# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
file_one = "Output/WeatherCheck.csv"
file_one_df = pd.read_csv(file_one, encoding="utf-8")
file_one_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,rikitea,78.0,PF,1586638912,74.0,-23.12,-134.97,77.04,1.34
1,huron,75.0,US,1586638912,37.0,41.15,-82.55,57.00,12.75
2,carolina,20.0,PR,1586638745,70.0,18.38,-65.96,88.00,16.11
3,westport,75.0,US,1586638878,22.0,41.14,-73.36,55.99,12.75
4,dire dawa,51.0,ET,1586638899,46.0,9.59,41.87,73.35,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [17]:
locations = file_one_df[["Latitude", "Longitude"]].astype(float)
humidity = file_one_df["Humidity"].astype(float)

In [26]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.


vacation_spots = file_one_df[file_one_df['Max Temp'] > 70]
vacation_spots = vacation_spots[vacation_spots['Max Temp'] < 80]
vacation_spots = vacation_spots[vacation_spots['Wind Speed'] < 10]
vacation_spots = vacation_spots[vacation_spots['Cloudiness'] == 0]
vacation_spots

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
95,periyanayakkanpalaiyam,0.0,IN,1586638948,54.0,11.15,76.93,76.51,2.04
111,mariental,0.0,NaN,1586638953,18.0,-24.63,17.97,76.82,9.60
145,pisco,0.0,PE,1586638965,30.0,-13.70,-76.22,75.00,8.05
203,dakar,0.0,SN,1586638829,83.0,14.69,-17.44,77.00,6.93
228,namibe,0.0,AO,1586638785,61.0,-15.20,12.15,77.59,3.62
235,ambilobe,0.0,MG,1586638998,84.0,-13.20,49.05,73.67,4.72
241,pozo colorado,0.0,PY,1586639001,37.0,-23.49,-58.80,78.71,2.48
265,antsohihy,0.0,MG,1586639008,73.0,-14.88,47.99,76.64,8.37
356,gbarnga,0.0,LR,1586639040,83.0,7.00,-9.47,75.65,3.60
383,maracaju,0.0,BR,1586639049,46.0,-21.61,-55.17,75.63,1.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [122]:
vacation_spots["Hotel Name"] = ''
vacation_spots = vacation_spots.reset_index()
vacation_spots = vacation_spots[['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Latitude', 'Longitude', 'Max Temp', 
                                'Wind Speed', 'Hotel Name']]
vacation_spots = pd.DataFrame(vacation_spots)
hotel_df = []
import json
vacation_spots

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,periyanayakkanpalaiyam,0.0,IN,1586638948,54.0,11.15,76.93,76.51,2.04,
1,mariental,0.0,NaN,1586638953,18.0,-24.63,17.97,76.82,9.60,
2,pisco,0.0,PE,1586638965,30.0,-13.70,-76.22,75.00,8.05,
3,dakar,0.0,SN,1586638829,83.0,14.69,-17.44,77.00,6.93,
4,namibe,0.0,AO,1586638785,61.0,-15.20,12.15,77.59,3.62,
5,ambilobe,0.0,MG,1586638998,84.0,-13.20,49.05,73.67,4.72,
6,pozo colorado,0.0,PY,1586639001,37.0,-23.49,-58.80,78.71,2.48,
7,antsohihy,0.0,MG,1586639008,73.0,-14.88,47.99,76.64,8.37,
8,gbarnga,0.0,LR,1586639040,83.0,7.00,-9.47,75.65,3.60,
9,maracaju,0.0,BR,1586639049,46.0,-21.61,-55.17,75.63,1.79,


In [124]:
for index, row in vacation_spots.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latit = vacation_spots['Latitude']
    longi = vacation_spots['Longitude']
    city = vacation_spots['City']
    
    address = f"{latit}, {longi}"
    
    params = {
    "location": address,
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": g_key}
    
    print(f"Retrieving Results for Index {index}: {city} at {address}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"First hotel in {city}  is {results[0]['name']}.") 
        vacation_spots.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  
    print("------------")

Retrieving Results for Index 0: 0     periyanayakkanpalaiyam
1                  mariental
2                      pisco
3                      dakar
4                     namibe
5                   ambilobe
6              pozo colorado
7                  antsohihy
8                    gbarnga
9                   maracaju
10                walvis bay
11                    jardim
12                    razole
13                    amahai
14                      tura
15                   artigas
16       santiago del estero
Name: City, dtype: object at 0     11.15
1    -24.63
2    -13.70
3     14.69
4    -15.20
5    -13.20
6    -23.49
7    -14.88
8      7.00
9    -21.61
10   -22.96
11   -21.48
12    16.48
13    -3.33
14    25.52
15   -30.40
16   -27.80
Name: Latitude, dtype: float64, 0      76.93
1      17.97
2     -76.22
3     -17.44
4      12.15
5      49.05
6     -58.80
7      47.99
8      -9.47
9     -55.17
10     14.51
11    -56.14
12     81.83
13    128.92
14     90.22
15    -56.47
16

Missing field/result... skipping.
------------
Retrieving Results for Index 8: 0     periyanayakkanpalaiyam
1                  mariental
2                      pisco
3                      dakar
4                     namibe
5                   ambilobe
6              pozo colorado
7                  antsohihy
8                    gbarnga
9                   maracaju
10                walvis bay
11                    jardim
12                    razole
13                    amahai
14                      tura
15                   artigas
16       santiago del estero
Name: City, dtype: object at 0     11.15
1    -24.63
2    -13.70
3     14.69
4    -15.20
5    -13.20
6    -23.49
7    -14.88
8      7.00
9    -21.61
10   -22.96
11   -21.48
12    16.48
13    -3.33
14    25.52
15   -30.40
16   -27.80
Name: Latitude, dtype: float64, 0      76.93
1      17.97
2     -76.22
3     -17.44
4      12.15
5      49.05
6     -58.80
7      47.99
8      -9.47
9     -55.17
10     14.51
11    -56.14
12     

Missing field/result... skipping.
------------
Retrieving Results for Index 16: 0     periyanayakkanpalaiyam
1                  mariental
2                      pisco
3                      dakar
4                     namibe
5                   ambilobe
6              pozo colorado
7                  antsohihy
8                    gbarnga
9                   maracaju
10                walvis bay
11                    jardim
12                    razole
13                    amahai
14                      tura
15                   artigas
16       santiago del estero
Name: City, dtype: object at 0     11.15
1    -24.63
2    -13.70
3     14.69
4    -15.20
5    -13.20
6    -23.49
7    -14.88
8      7.00
9    -21.61
10   -22.96
11   -21.48
12    16.48
13    -3.33
14    25.52
15   -30.40
16   -27.80
Name: Latitude, dtype: float64, 0      76.93
1      17.97
2     -76.22
3     -17.44
4      12.15
5      49.05
6     -58.80
7      47.99
8      -9.47
9     -55.17
10     14.51
11    -56.14
12    

In [121]:
vacation_spots

#hotels_data = response.json()
#print(json.dumps(hotels_data, indent=4, sort_keys=True))

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,periyanayakkanpalaiyam,0.0,IN,1586638948,54.0,11.15,76.93,76.51,2.04,
1,mariental,0.0,NaN,1586638953,18.0,-24.63,17.97,76.82,9.60,
2,pisco,0.0,PE,1586638965,30.0,-13.70,-76.22,75.00,8.05,
3,dakar,0.0,SN,1586638829,83.0,14.69,-17.44,77.00,6.93,
4,namibe,0.0,AO,1586638785,61.0,-15.20,12.15,77.59,3.62,
5,ambilobe,0.0,MG,1586638998,84.0,-13.20,49.05,73.67,4.72,
6,pozo colorado,0.0,PY,1586639001,37.0,-23.49,-58.80,78.71,2.48,
7,antsohihy,0.0,MG,1586639008,73.0,-14.88,47.99,76.64,8.37,
8,gbarnga,0.0,LR,1586639040,83.0,7.00,-9.47,75.65,3.60,
9,maracaju,0.0,BR,1586639049,46.0,-21.61,-55.17,75.63,1.79,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map